In [9]:
import os

import dotenv
import pandas as pd


In [10]:
# !pip install -e ../../../pipeline

from data_inclusion.scripts.tasks import grist


In [11]:
dotenv.load_dotenv(dotenv.find_dotenv())

GRIST_API_TOKEN = os.environ["GRIST_API_TOKEN"]
GRIST_API_URL = "https://grist.incubateur.anct.gouv.fr/api"
WORKSPACE_ID = "1640"
DOCUMENT_NAME = "monenfant"


In [12]:
grist_client = grist.GristClient(base_url=GRIST_API_URL, token=GRIST_API_TOKEN)

document_id = grist_client.create_document(
    workspace_id=WORKSPACE_ID, document_name=DOCUMENT_NAME
)


In [13]:
grist_client.create_table(
    document_id=document_id,
    table_name="Typologie_de_structures",
    columns=[
        {"id": "value", "fields": {"label": "valeur", "type": "Text"}},
        {"id": "label", "fields": {"label": "label", "type": "Text"}},
    ],
)


{'tables': [{'id': 'Typologie_de_structures'}]}

In [14]:
typologies_df = pd.read_csv(
    "../../../pipeline/dbt/seeds/schema/typologies_de_structures.csv",
    dtype=str,
)


In [15]:
grist_client.add_records(
    document_id=document_id,
    table_id="Typologie_de_structures",
    records=[
        {"fields": typologie_dict}
        for typologie_dict in typologies_df[["value", "label"]].to_dict(
            orient="records"
        )
    ],
)


<Response [200]>

In [16]:
grist_client.create_table(
    document_id=document_id,
    table_name="Structures",
    columns=[
        {
            "id": "id",
            "fields": {"label": "id", "type": "Text"},
        },
        {
            "id": "siret",
            "fields": {"label": "siret", "type": "Text"},
        },
        {
            "id": "rna",
            "fields": {"label": "rna", "type": "Text"},
        },
        {
            "id": "nom",
            "fields": {"label": "nom", "type": "Text"},
        },
        {
            "id": "commune",
            "fields": {"label": "commune", "type": "Text"},
        },
        {
            "id": "code_postal",
            "fields": {"label": "code_postal", "type": "Text"},
        },
        {
            "id": "code_insee",
            "fields": {"label": "code_insee", "type": "Text"},
        },
        {
            "id": "adresse",
            "fields": {"label": "adresse", "type": "Text"},
        },
        {
            "id": "complement_adresse",
            "fields": {"label": "complement_adresse", "type": "Text"},
        },
        {
            "id": "longitude",
            "fields": {"label": "longitude", "type": "Numeric"},
        },
        {
            "id": "latitude",
            "fields": {"label": "latitude", "type": "Numeric"},
        },
        {
            "id": "typologie",
            "fields": {
                "label": "typologie",
                "type": "Ref:Typologie_de_structures",
            },
        },
        {
            "id": "telephone",
            "fields": {"label": "telephone", "type": "Text"},
        },
        {
            "id": "courriel",
            "fields": {"label": "courriel", "type": "Text"},
        },
        {
            "id": "site_web",
            "fields": {"label": "site_web", "type": "Text"},
        },
        {
            "id": "presentation_resume",
            "fields": {"label": "presentation_resume", "type": "Text"},
        },
        {
            "id": "presentation_detail",
            "fields": {"label": "presentation_detail", "type": "Text"},
        },
        {
            "id": "source",
            "fields": {"label": "source", "type": "Text"},
        },
        {
            "id": "date_maj",
            "fields": {
                "label": "date_maj",
                "type": "DateTime:Europe/Paris",
            },
        },
        {
            "id": "antenne",
            "fields": {"label": "antenne", "type": "Bool"},
        },
        {
            "id": "lien_source",
            "fields": {"label": "lien_source", "type": "Text"},
        },
        {
            "id": "horaires_ouverture",
            "fields": {"label": "horaires_ouverture", "type": "Text"},
        },
        {
            "id": "accessibilite",
            "fields": {"label": "accessibilite", "type": "Text"},
        },
        {
            "id": "labels_nationaux",
            "fields": {"label": "labels_nationaux", "type": "Text"},
        },
        {
            "id": "labels_autres",
            "fields": {"label": "labels_autres", "type": "Text"},
        },
        {
            "id": "thematiques",
            "fields": {"label": "thematiques", "type": "Text"},
        },
    ],
)
# TODO: services


{'tables': [{'id': 'Structures'}]}